**Save and Restore a model**

In [30]:
!pip install h5py pyyaml

In [31]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

Define a Model

Load MNIST data

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [33]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Save Checkpoints During training.

tf.keras.callbacks.ModelCheckpoint

In [34]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 704/1000 [====================>.........] - ETA: 0s - loss: 1.3692 - accuracy: 0.6151
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 687us/sample - loss: 1.1891 - accuracy: 0.6580 - val_loss: 0.7075 - val_accuracy: 0.7910
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.3944 - accuracy: 0.9036
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 245us/sample - loss: 0.4148 - accuracy: 0.8900 - val_loss: 0.5302 - val_accuracy: 0.8310
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.3237 - accuracy: 0.9089
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 249us/sample - loss: 0.3032 - accuracy: 0.9200 - val_loss: 0.5065 - val_accuracy: 0.8400
Epoch 4/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.1976 - accuracy: 0.9503
Epoch

Check Directory

In [35]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Now, Build a fresh untrained Model

In [36]:
mmodel = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained Model, Accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Now load the model weights stored earlier

In [37]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained Model, Accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Checking CallBack options

In [38]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create Checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weight_only=True,
    # Save period every 5 epochs
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images, train_labels, epochs=50,
          callbacks=[cp_callback], validation_data=(test_images, test_labels),
          verbose=0)

INFO:tensorflow:Assets written to: training_2/cp-0005.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0010.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0015.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0020.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0025.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0030.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0035.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0040.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0045.ckpt/assets
INFO:tensorflow:Assets written to: training_2/cp-0050.ckpt/assets


Latest Checkpoint

In [39]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0000.ckpt'

In [40]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Manually Save weights

In [41]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Save Entire Model: Weights and optimizaer informations

As HDF5: Hierarchical Data File

In [42]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 763us/sample - loss: 1.1107 - accuracy: 0.6850
Epoch 2/5
1000/1000 [==============================] - 0s 177us/sample - loss: 0.4100 - accuracy: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 160us/sample - loss: 0.2802 - accuracy: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 164us/sample - loss: 0.2138 - accuracy: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 164us/sample - loss: 0.1524 - accuracy: 0.9660


Recreate the model from HDF5

In [43]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_22 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [44]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

This technique saves everything:

    The weight values
    The model's configuration(architecture)
    The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.


Conclusion